In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.regression.linear_model import OLS
import itertools

# Step 1: Data Acquisition
def get_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    for ticker in tickers:
        temp = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
        data[ticker] = temp
    return data.dropna()

# Example tickers (large-cap stocks from various sectors)
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'FB', 'JNJ', 'WMT', 'V', 'PG', 'DIS', 'KO', 'PEP']

start_date = '2020-01-01'
end_date = '2024-06-02'

data = get_data(tickers, start_date, end_date)

# Step 2: Test All Pairs for Cointegration
def test_cointegration(series1, series2):
    series1 = np.log(series1)
    series2 = np.log(series2)
    result = coint(series1, series2)
    return result[1]  # p-value

pairs = []
for (ticker1, ticker2) in itertools.combinations(tickers, 2):
    p_value = test_cointegration(data[ticker1], data[ticker2])
    if p_value < 0.05:
        pairs.append((ticker1, ticker2, p_value))

# Sort pairs by p-value and select top 5
pairs.sort(key=lambda x: x[2])
top_pairs = pairs[:5]

print("Top Cointegrated Pairs:")
for pair in top_pairs:
    print(f"{pair[0]} - {pair[1]}: p-value = {pair[2]}")

# Step 3-8: Apply Trading Strategy to Each Pair
pair_returns = {}

for pair1, pair2, _ in top_pairs:
    # Calculate Spread
    series1 = np.log(data[pair1])
    series2 = np.log(data[pair2])
    model = OLS(series1, series2).fit()
    spread = series1 - model.params[0] * series2
    hedge_ratio = model.params[0]

    # Calculate Z-score
    z_score = (spread - spread.mean()) / spread.std()

    # Set Trading Signals
    z_entry_threshold = 2
    z_exit_threshold = 0

    position = np.where(z_score > z_entry_threshold, -1,
                      np.where(z_score < -z_entry_threshold, 1, 0))

    # Apply exit conditions
    position = np.where((position == 1) & (z_score > -z_exit_threshold), 0, position)
    position = np.where((position == -1) & (z_score < z_exit_threshold), 0, position)

    # Fill forward to hold positions
    position = pd.Series(position, index=data.index).fillna(method='ffill')

    # Calculate Returns
    returns1 = np.log(data[pair1] / data[pair1].shift(1))
    returns2 = np.log(data[pair2] / data[pair2].shift(1))

    # Pair trading returns
    pair_return = position.shift(1) * (returns1 - hedge_ratio * returns2)
    cumulative_return = (1 + pair_return).cumprod()

    pair_returns[f"{pair1}-{pair2}"] = cumulative_return

# Step 9: Visualizations
# Plot cumulative returns for each pair
plt.figure(figsize=(12, 6))
for pair, returns in pair_returns.items():
    plt.plot(returns.index, returns, label=pair)
plt.title('Cumulative Returns - All Pairs')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.grid()
plt.legend()
plt.show()

# Plot z-scores for the best pair
best_pair = top_pairs[0]
pair1, pair2, _ = best_pair

series1 = np.log(data[pair1])
series2 = np.log(data[pair2])
model = OLS(series1, series2).fit()
spread = series1 - model.params[0] * series2
z_score = (spread - spread.mean()) / spread.std()

plt.figure(figsize=(12, 6))
plt.plot(z_score.index, z_score)
plt.axhline(y=z_entry_threshold, color='g', linestyle='--', label='Entry/Exit Threshold')
plt.axhline(y=-z_entry_threshold, color='g', linestyle='--')
plt.axhline(y=z_exit_threshold, color='r', linestyle='--')
plt.axhline(y=-z_exit_threshold, color='r', linestyle='--')
plt.axhline(y=0, color='black')
plt.title(f'Z-Score - {pair1} vs {pair2}')
plt.xlabel('Date')
plt.ylabel('Z-Score')
plt.grid()
plt.legend()
plt.show()

# Step 10: Performance Metrics
print("\nPerformance Metrics:")
for pair, returns in pair_returns.items():
    total_return = returns.iloc[-1] - 1
    pair_returns_series = returns.pct_change().dropna()
    annualized_return = (total_return + 1) ** (252 / len(returns)) - 1
    annualized_volatility = pair_returns_series.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_volatility
    max_drawdown = (returns / returns.cummax() - 1).min()

    print(f"\n{pair}:")
    print(f"  Total Return: {total_return:.2%}")
    print(f"  Annualized Return: {annualized_return:.2%}")
    print(f"  Annualized Volatility: {annualized_volatility:.2%}")
    print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"  Max Drawdown: {max_drawdown:.2%}")

# Step 11: Portfolio-level Analysis
portfolio_returns = pd.DataFrame(pair_returns).mean(axis=1)
total_return = portfolio_returns.iloc[-1] - 1
portfolio_returns_series = portfolio_returns.pct_change().dropna()
annualized_return = (total_return + 1) ** (252 / len(portfolio_returns)) - 1
annualized_volatility = portfolio_returns_series.std() * np.sqrt(252)
sharpe_ratio = annualized_return / annualized_volatility
max_drawdown = (portfolio_returns / portfolio_returns.cummax() - 1).min()

print("\nPortfolio Performance:")
print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown:.2%}")

# Plot portfolio cumulative returns
plt.figure(figsize=(12, 6))
plt.plot(portfolio_returns.index, portfolio_returns)
plt.title('Portfolio Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.grid()
plt.show()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


ValueError: zero-size array to reduction operation maximum which has no identity